In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [31]:
from itertools import islice
from pathlib import Path
from IPython.display import Audio
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import glob
import os
import random
import torchaudio
import scipy
from scipy.fftpack import fft
from scipy.signal import get_window

In [150]:
base = Path("/home/jupyter/rob/TIMIT/timit")
path_data = base/"TRAIN"
path_phoneme = base/"PHONEMES"

In [156]:
if (not os.path.exists(path_phoneme)):
    os.mkdir(path_phoneme)

In [151]:
def getPhonemeListFromPhonemeFile(phoneme_filename):
    with open(phoneme_filename, 'r') as f:
        phoneme_data = f.read()
    #first number is start time, second is endtime, last group is the phonetic symbol (NIST format)
    phoneme_list = re.findall("(\d+) (\d+) ([^\\n]+)", phoneme_data)
    return phoneme_list

In [157]:
def chopWavFileToPhonemes(wav_filename):
    clip, sample_rate = torchaudio.load(wav_filename)
    clip = clip.squeeze()
    phoneme_datafile = wav_filename.replace(".WAV", ".PHN")
    phoneme_list = getPhonemeListFromPhonemeFile(phoneme_datafile)
    for start_index, end_index, phoneme in phoneme_list:
        specific_phoneme_path = path_phoneme/phoneme
        if (not os.path.exists(specific_phoneme_path)):
            os.mkdir(specific_phoneme_path)
        save_portion = clip[int(start_index):int(end_index)]
        phoneme_savefile = ('-'.join(wav_filename.split('/')[-3:-1]) + f'-{random.randint(0,2**20)}.WAV')
        torchaudio.save(str(specific_phoneme_path/phoneme_savefile), save_portion, sample_rate)

In [158]:
def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

In [159]:
# takes 1-2 minutes to run
def chopAllWavToPhonemes():
    count = 0
    wav_files = [fpath for fpath in absoluteFilePaths(path_data) if fpath.split('.')[-1] == 'WAV']
    for wav_file in wav_files:
        chopWavFileToPhonemes(wav_file)

In [160]:
chopAllWavToPhonemes()